In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext


sc = SparkContext('local', 'Spark SQL') 
spark = SQLContext(sc)
from pyspark.sql.functions import explode, col

In [13]:
source_df = spark.read.option("multiline", "true").json("prize.json")
source_df.printSchema()
source_df.show(5, False)

root
 |-- prizes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- laureates: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- firstname: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- motivation: string (nullable = true)
 |    |    |    |    |-- share: string (nullable = true)
 |    |    |    |    |-- surname: string (nullable = true)
 |    |    |-- overallMotivation: string (nullable = true)
 |    |    |-- year: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
prizes = source_df.select(explode("prizes").alias("prizes"))
display(prizes)
prizes.printSchema()
prizes.show(5, False)

DataFrame[prizes: struct<category:string,laureates:array<struct<firstname:string,id:string,motivation:string,share:string,surname:string>>,overallMotivation:string,year:string>]

root
 |-- prizes: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- laureates: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- firstname: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- motivation: string (nullable = true)
 |    |    |    |-- share: string (nullable = true)
 |    |    |    |-- surname: string (nullable = true)
 |    |-- overallMotivation: string (nullable = true)
 |    |-- year: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prizes                                                          

In [15]:
prizes_laureates = prizes.select(
col("prizes.category").alias("prizes_category"),
explode("prizes.laureates").alias("prizes_laureates"),
col("prizes.overallMotivation").alias("prizes_overallMotivation"),
col("prizes.year").alias("prizes_year")
)
display(prizes_laureates)
prizes_laureates.printSchema()
prizes_laureates.show(5, False)

DataFrame[prizes_category: string, prizes_laureates: struct<firstname:string,id:string,motivation:string,share:string,surname:string>, prizes_overallMotivation: string, prizes_year: string]

root
 |-- prizes_category: string (nullable = true)
 |-- prizes_laureates: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- motivation: string (nullable = true)
 |    |-- share: string (nullable = true)
 |    |-- surname: string (nullable = true)
 |-- prizes_overallMotivation: string (nullable = true)
 |-- prizes_year: string (nullable = true)

+---------------+--------------------------------------------------------------------------------------------+------------------------+-----------+
|prizes_category|prizes_laureates                                                                            |prizes_overallMotivation|prizes_year|
+---------------+--------------------------------------------------------------------------------------------+------------------------+-----------+
|chemistry      |[John, 976, "for the development of lithium-ion batteries", 3, Goodenough]                  |null                    |201

In [16]:
prizes_laureates_df = prizes_laureates.select(
col("prizes_category"),
col("prizes_overallMotivation"),
col("prizes_year"),
col("prizes_laureates.firstname").alias("prizes_laureates_firstname"),
col("prizes_laureates.id").alias("prizes_laureates_id"),
col("prizes_laureates.motivation").alias("prizes_laureates_motivation"),
col("prizes_laureates.share").alias("prizes_laureates_share"),
col("prizes_laureates.surname").alias("prizes_laureates_surname")    
)
display(prizes_laureates_df)
prizes_laureates_df.printSchema()


DataFrame[prizes_category: string, prizes_overallMotivation: string, prizes_year: string, prizes_laureates_firstname: string, prizes_laureates_id: string, prizes_laureates_motivation: string, prizes_laureates_share: string, prizes_laureates_surname: string]

root
 |-- prizes_category: string (nullable = true)
 |-- prizes_overallMotivation: string (nullable = true)
 |-- prizes_year: string (nullable = true)
 |-- prizes_laureates_firstname: string (nullable = true)
 |-- prizes_laureates_id: string (nullable = true)
 |-- prizes_laureates_motivation: string (nullable = true)
 |-- prizes_laureates_share: string (nullable = true)
 |-- prizes_laureates_surname: string (nullable = true)



In [17]:
prizes_laureates_df.show(5, False)

+---------------+------------------------+-----------+--------------------------+-------------------+---------------------------------------------------------------+----------------------+------------------------+
|prizes_category|prizes_overallMotivation|prizes_year|prizes_laureates_firstname|prizes_laureates_id|prizes_laureates_motivation                                    |prizes_laureates_share|prizes_laureates_surname|
+---------------+------------------------+-----------+--------------------------+-------------------+---------------------------------------------------------------+----------------------+------------------------+
|chemistry      |null                    |2019       |John                      |976                |"for the development of lithium-ion batteries"                 |3                     |Goodenough              |
|chemistry      |null                    |2019       |M. Stanley                |977                |"for the development of lithium-ion batteri

In [18]:
prizes_laureates_df.registerTempTable("nobel_prize")

# Nobel Prize winners more than once

In [27]:
spark.sql("""
select prizes_laureates_firstname,prizes_laureates_surname, count(*)  
from nobel_prize 
group by prizes_laureates_firstname, prizes_laureates_surname 
having count(*) > 1 
order by 3 desc 
""").show(20, False)

+-----------------------------------------------------------+------------------------+--------+
|prizes_laureates_firstname                                 |prizes_laureates_surname|count(1)|
+-----------------------------------------------------------+------------------------+--------+
|International Committee of the Red Cross                   |null                    |3       |
|Frederick                                                  |Sanger                  |2       |
|Marie                                                      |Curie                   |2       |
|Linus                                                      |Pauling                 |2       |
|Office of the United Nations High Commissioner for Refugees|null                    |2       |
|John                                                       |Bardeen                 |2       |
+-----------------------------------------------------------+------------------------+--------+

